In [1]:
#Checking if GPU is running or not

!nvidia-smi

Mon Nov 20 03:34:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00


In [2]:
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.1 MB/s eta 0:00:00


In [5]:
!pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [9]:
model_checkpoint = "persiannlp/mt5-base-parsinlu-opus-translation_fa_en"


## Helsinki-NLP/opus-mt-en-hi model

source: https://huggingface.co/Helsinki-NLP/opus-mt-en-hi



# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [10]:
raw_datasets = load_dataset("persiannlp/parsinlu_translation_en_fa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1621665 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/55c737abe7176f5400ad633e32bac2b7e4937e92cebfe8d5e532db9649dd6f04


Generating test split:   0%|          | 0/48359 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/9844c851544d13187c3f4101de8cc9e13f48c842d93252a10515bf07fb943575


Generating validation split:   0%|          | 0/2137 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/4db460fb756175358599a5f7ad7131c3e5fe381b98a8918c0281bc4c6286fc8a


In [10]:
raw_datasets_1 = load_dataset("persiannlp/parsinlu_translation_en_fa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1621665 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/55c737abe7176f5400ad633e32bac2b7e4937e92cebfe8d5e532db9649dd6f04


Generating test split:   0%|          | 0/48359 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/9844c851544d13187c3f4101de8cc9e13f48c842d93252a10515bf07fb943575


Generating validation split:   0%|          | 0/2137 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/4db460fb756175358599a5f7ad7131c3e5fe381b98a8918c0281bc4c6286fc8a


In [13]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 1621665
    })
    test: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 48359
    })
    validation: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 2137
    })
})

In [12]:
raw_datasets_1

DatasetDict({
    train: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 1621665
    })
    test: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 48359
    })
    validation: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 2137
    })
})

In [14]:
raw_datasets['train'][10]

{'source': 'Unlike so many world countries, in Egyptian IDs, religion must be listed.',
 'targets': ['برخلاف بسیاری از کشورهای دنیا در کارت شناسایی\u200cهای مصر دین افراد باید ذکر شود'],
 'category': 'global_voices_en_fa'}

In [12]:
from datasets import load_dataset

def transform_example(example):
    return {
        "translation": {
            "en": example["source"],
            "fa": example["targets"][0]
        }
    }

#raw_datasets_1 = load_dataset("persiannlp/parsinlu_translation_en_fa")

new_datasets = raw_datasets.map(transform_example)

print(new_datasets)

Map:   0%|          | 0/1621665 [00:00<?, ? examples/s]

Map:   0%|          | 0/48359 [00:00<?, ? examples/s]

Map:   0%|          | 0/2137 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'targets', 'category', 'translation'],
        num_rows: 1621665
    })
    test: Dataset({
        features: ['source', 'targets', 'category', 'translation'],
        num_rows: 48359
    })
    validation: Dataset({
        features: ['source', 'targets', 'category', 'translation'],
        num_rows: 2137
    })
})


In [15]:
raw_dataset = {}

for split in new_datasets.keys():
    raw_dataset[split] = [example['translation'] for example in new_datasets[split]]

In [16]:
from datasets import Dataset, DatasetDict

raw_dataset = DatasetDict({
    split: Dataset.from_pandas(new_datasets[split].remove_columns(["source", "targets", "category"]).to_pandas())
    for split in new_datasets.keys()
})

In [19]:
raw_datasets['train'][10]

{'translation': {'en': 'The color and opacity of the highlight fill.',
  'hi': 'हाइलाइट किया गया भराई का रंग और पारदर्शिता। '}}

In [17]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1621665
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 48359
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2137
    })
})

#Preprocessing the data

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [24]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["हाइलाइट किया गया भराई का रंग और पारदर्शिता।"]))

{'input_ids': [[10866, 78, 115, 870, 4011, 24, 593, 9, 16084, 40, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "fa"

prefix = "translate English to Persian: "
def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
preprocess_function(raw_dataset["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[59593, 16546, 1662, 16932, 455, 32213, 4325, 39630, 138249, 5546, 470, 11039, 162183, 267, 259, 85336, 1638, 25059, 702, 259, 54920, 261, 790, 17564, 1638, 2262, 2603, 259, 89555, 359, 53952, 1258, 213192, 302, 336, 728, 1652, 1476, 1], [1669, 31637, 345, 751, 259, 82519, 259, 172784, 305, 259, 68675, 533, 3153, 345, 2478, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3923, 6062, 10653, 406, 14273, 572, 556, 45039, 259, 37365, 274, 134867, 406, 271, 6423, 695, 6864, 16905, 1440, 695, 47536, 941, 259, 2394, 4725, 8180, 2154, 548, 9234, 2657, 916, 404, 3691, 6864, 12988, 633, 436, 274, 134867, 406, 271, 7917, 3716, 260, 1], [4424, 695, 259, 6513, 406, 259, 67914, 50771, 343, 14727, 1875, 341, 259, 11890, 406, 934, 695, 4424, 259, 36345, 406, 27424, 9438, 8376, 343, 766, 50966, 3716, 260, 1]]}

In [22]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1621665 [00:00<?, ? examples/s]

Map:   0%|          | 0/48359 [00:00<?, ? examples/s]

Map:   0%|          | 0/2137 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [24]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="en_to_fa_translation_machine",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss


# Model Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
input_text  = "My name Bappy. My youtube channel name is DSwithBappy"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179  4598 14918   130     5     2   500   118  2136  4186
   4850   179     5  4404     3   906     3   906     3    12     3  1370
      5    40     0 61949]], shape=(1, 28), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

मेरा नाम बाप्पी है, मेरा आपटबे चैनल नाम है डी.के.के. से. आता है।


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
